# FABRIC API: Renew a Slice Reservation


## Configure the environment

In [ ]:
import os

# If you are using the FABRIC JupyterHub, the following three evnrionment vars
# were automatically provided when you logged in.
#os.environ['FABRIC_CREDMGR_HOST']='cm.fabric-testbed.net'
#os.environ['FABRIC_ORCHESTRATOR_HOST']='orchestrator.fabric-testbed.net'
#os.environ['FABRIC_TOKEN_LOCATION']=os.environ['HOME']+'/work/fabric_token.json'

# Bastion IPs
os.environ['FABRIC_BASTION_HOST'] = 'bastion-1.fabric-testbed.net'

# Set your Bastion username and private key
os.environ['FABRIC_BASTION_USERNAME']='pruth'
os.environ['FABRIC_BASTION_KEY_LOCATION']=os.environ['HOME']+'/work/.ssh/id_rsa_fabric'

# Set the keypair FABRIC will install in your slice.
os.environ['FABRIC_SLICE_PRIVATE_KEY_FILE']=os.environ['HOME']+'/.ssh/id_rsa'
os.environ['FABRIC_SLICE_PUBLIC_KEY_FILE']=os.environ['HOME']+'/.ssh/id_rsa.pub'

# If your private key uses a passphrase, set the passphrase
#from getpass import getpass
#print('Please input private key passphrase.')
#os.environ['FABRIC_SLICE_PRIVATE_KEY_PASSPHRASE']=getpass()

## Setup the Experiment

#### Import FABRIC API

In [ ]:
import json
import traceback
from fabrictestbed_extensions.fablib.fablib import fablib

## Configure Slice Parameters



In [ ]:
slice_name='MySlice'

## Get and Renew the Slice

You slice is in the list of all your slices. You can loop through the list of slices to get the slice. Python has a standard tool to filter lists. Try using a lambda function to filter out your slice using its name.

In [ ]:
import datetime

#Set end host to now plus 1 day
end_date = (datetime.datetime.now() + datetime.timedelta(days=6)).strftime("%Y-%m-%d %H:%M:%S")

try:
    slice = fablib.get_slice(name=slice_name)
    slice.renew(end_date)
except Exception as e:
    print(f"Get Slices Fail: {e}")
    traceback.print_exc()

## Check New Lease End Date

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
    print(f"Lease End         : {slice.get_lease_end()}")
       
except Exception as e:
    print(f"Fail: {e}")
    traceback.print_exc()